In [1]:
import os

import numpy as np
import pandas as pd


import re

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence



2023-04-13 07:24:35.071255: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 07:24:36.107995: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-04-13 07:24:36.108095: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [13]:
tf.__version__

'2.11.0'

In [10]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [12]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2023-04-13 08:00:01.862505: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 08:00:01.863264: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 08:00:01.863699: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 08:00:01.864127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 08:00:01.864549: I tensorflow/compiler/xla/stream_executo

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15788773136376394805
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15332737024
 locality {
   bus_id: 1
   links {
     link {
       device_id: 1
       type: "StreamExecutor"
       strength: 1
     }
     link {
       device_id: 2
       type: "StreamExecutor"
       strength: 1
     }
     link {
       device_id: 3
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 6111834064553688664
 physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"
 xla_global_id: 416903419,
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 15332737024
 locality {
   bus_id: 1
   links {
     link {
       type: "StreamExecutor"
       strength: 1
     }
     link {
       device_id: 2
       type: "StreamExecutor"
       strength: 1
     }
     link {
       device_id: 3
       type: "S

In [ ]:
# padding 514(inclusive to special token), 1240?
# tokenizer.json
# config.json
# 더 긴 input
# bigberd, longformer
# google/bigbird-roberta-base
# running rate 최대한 줄여서

In [2]:
train = pd.read_csv('./data/train.csv', header=None, names=['text','time','label'])
test = pd.read_csv('./data/test.csv', header=None, names=['text','time','label'])


In [3]:
X_train_list = train['text'].tolist()
X_test_list = test['text'].tolist()
y_train = train['label'].tolist()
y_test = test['label'].tolist()

In [22]:
# def rexpreprocess(sent):
#     sent = re.sub(r"([?.!,])", r" \1", sent)
#     sent = re.sub(r"[^a-zA-Z!,.?]+", r" ", sent)
#     sent = re.sub(r"\s+", " ", sent)
#     return sent

In [23]:
# X_train_list = [rexpreprocess(sent) for sent in X_train_list]
# X_test_list = [rexpreprocess(sent) for sent in X_test_list]


In [24]:
# X_train_drop = [i for i,e in enumerate(X_train_list) if e == ' ']
# X_test_drop = [i for i,e in enumerate(X_test_list) if e == ' ']
# len(X_train_drop), len(X_test_drop)

(75, 144)

In [25]:
# for i in sorted(X_train_drop, reverse=True):
#     del X_train_list[i]
#     del y_train[i]
# for i in sorted(X_test_drop, reverse=True):
#     del X_test_list[i]
#     del y_test[i]

In [4]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer #,BigBirdTokenizer, BigBirdTokenizerFast

In [5]:
# M = 'bert-base-uncased'
# M = "cardiffnlp/twitter-roberta-base-hate"
# M = 'distilbert-base-uncased'
M = 'allenai/longformer-base-4096'

In [11]:
# import sentencepiece as spm

In [77]:
# tokenizer = AutoTokenizer.from_pretrained("google/bigbird-roberta-base")

In [6]:
tokenizer = AutoTokenizer.from_pretrained(M) # do_lower_case =True


In [7]:
X_train = tokenizer(X_train_list, truncation=True, padding=True)
# X_test = tokenizer(X_test_list, truncation=True, padding=True)

In [13]:
X_train

{'input_ids': [[0, 725, 3019, 479, 20280, 479, 45676, 62, 17487, 45, 10, 4866, 479, 47, 17487, 276, 479, 145, 22414, 479, 256, 50, 856, 17487, 274, 479, 211, 29765, 2156, 38, 10, 11474, 5030, 626, 3668, 1085, 19, 127, 183, 12035, 2494, 2682, 15, 18220, 479, 256, 259, 479, 1801, 300, 184, 31, 983, 1805, 479, 255, 7651, 479, 5534, 2156, 3035, 479, 3664, 631, 17487, 11380, 479, 10, 28127, 784, 17487, 272, 16037, 657, 167, 479, 2156, 6467, 479, 8, 2512, 17487, 23129, 14223, 479, 2530, 479, 30857, 7, 6467, 479, 54, 5090, 3023, 495, 85, 10, 11474, 579, 2579, 2156, 965, 10, 11474, 326, 24, 17487, 8976, 479, 14947, 2254, 3918, 479, 6834, 2376, 47, 15, 384, 17421, 17487, 2997, 17487, 939, 10, 11474, 475, 12483, 187, 47, 439, 15, 10, 10, 11474, 284, 10, 11474, 1805, 181, 11380, 479, 1801, 303, 42, 1082, 10, 367, 360, 536, 479, 8976, 2156, 384, 17421, 479, 39530, 7, 5, 1086, 39809, 9624, 631, 39809, 20560, 5373, 5, 2172, 47, 972, 2156, 965, 10, 11474, 326, 24, 17487, 85, 10, 11474, 579, 57, 2295,

In [20]:
X_train[0]

Encoding(num_tokens=1787, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [75]:
len(X_train['input_ids'])

100

In [ ]:
tokenizer.tokenize(X_train_list[0])

In [ ]:
X_train.keys()

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train),
    y_train
))

# val_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(X_test),
#     y_test
# ))

2023-04-13 07:34:21.437400: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 07:34:21.437834: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 07:34:21.438221: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 07:34:21.438570: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 07:34:21.591098: I tensorflow/compiler/xla/stream_executo

In [91]:
id2label = {0: "Normal", 1: "Problem"}
label2id = {"Normal": 0, "Problem": 1}

In [92]:
model = TFAutoModelForSequenceClassification.from_pretrained(M, num_labels=2, id2label=id2label, label2id=label2id)#,from_pt=True 

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [93]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) #learning_rate=5e-5
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

In [94]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# callback_earlystop = EarlyStopping(
#     monitor="val_accuracy", 
#     min_delta=0.001,
#     patience=2)



In [97]:
model.fit(
    train_dataset.shuffle(len(X_train['input_ids'])).batch(16), epochs=1, batch_size=16,
    # validation_data = val_dataset.shuffle(len(X_test['input_ids'])).batch(32),
    # callbacks = [callback_earlystop]
)

2023-04-11 06:01:51.199483: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.17GiB (rounded to 1258291200)requested by op tf_roberta_for_sequence_classification_2/roberta/encoder/layer_._1/attention/self/Softmax
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-04-11 06:01:51.199595: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-04-11 06:01:51.199608: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 186, Chunks in use: 184. 46.5KiB allocated for chunks. 46.0KiB in use in bin. 849B client-requested in use in bin.
2023-04-11 06:01:51.199617: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 1, Chunks in use: 1. 512B allocated for chunks. 512B in use in bin. 400B client-requeste

ResourceExhaustedError: Graph execution error:

Detected at node 'tf_roberta_for_sequence_classification_2/roberta/encoder/layer_._1/attention/self/Softmax' defined at (most recent call last):
    File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
    

In [ ]:
X_test[0]

In [ ]:
from transformers import TextClassificationPipeline

# 로드하기

text_classifier = TextClassificationPipeline(
    tokenizer=tokenizer, 
    model=model, 
    framework='tf',
    return_all_scores=True
)

In [ ]:
text_classifier('hello')

In [ ]:
X_test[0].ids

In [46]:
# x = []
# for i in range(len(X_test)):
#     x.append(X_test[i].ids)

In [83]:
y_pred

TFSequenceClassifierOutput(loss=None, logits=array([[-0.2404954 ,  0.17563708],
       [-0.18905169,  0.07287241],
       [-0.18515065,  0.07454242],
       [-0.20931022,  0.10516676],
       [-0.21473381,  0.13949239],
       [-0.1882668 ,  0.07094803],
       [-0.21472913,  0.11845642],
       [-0.22111174,  0.11181909],
       [-0.20618683,  0.11706336],
       [-0.20797166,  0.1345488 ],
       [-0.20456326,  0.11369693],
       [-0.2124854 ,  0.12656903],
       [-0.20152879,  0.15915081],
       [-0.21589413,  0.13078247],
       [-0.23092696,  0.17367345],
       [-0.21246715,  0.09384146],
       [-0.19641286,  0.10546207],
       [-0.23839526,  0.18245608],
       [-0.21203507,  0.12826839],
       [-0.21698219,  0.1836533 ],
       [-0.24361895,  0.17344446],
       [-0.21808052,  0.15608954],
       [-0.19683066,  0.10102725],
       [-0.23143347,  0.15393963],
       [-0.18332177,  0.11096966],
       [-0.23538095,  0.17081156],
       [-0.20315206,  0.11347556],
       [-0

In [78]:
y_pred_score

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [ ]:
y_pred = model.predict(X_test['input_ids'])
y_pred_score = [np.argmax(e) for e in y_pred[0]]

In [84]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test,y_pred_score)
f1

0.019801980198019802